### 1 - Import libs

**Execute only**

In [ ]:
import time 
from pathlib import Path
from datetime import datetime as dt

import ee
import geemap
# initialize EE    
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

import helpers as h

### 2 - Basic Input Variables

**DEFINE YOUR INPUT VARIABLES**

In [ ]:
# Area of Interest
country = 'Kenya'   # country case
aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM0_NAME', country)) # here any feature collection can be selected

# Grid Size (in degrees)
grid_size = 1

# define an output file for CEO 
outfile = "/home/[SEPAL_USERNAME]/sbae.csv"

### 3 - Generate GRID and POINTS

**Execute only**

In [ ]:
# Generate the grid
grid, grid_fc = h.generate_grid(aoi, grid_size, grid_size, 0, 0)

# get a random point over each grid cell
randomInCell = grid_fc.map(h.get_random_point).filterBounds(aoi)

# get the centroid of each gridcell
centerInCell = grid_fc.map(h.get_center_point).filterBounds(aoi)

# stratified sampling based on ESA LC
imageCollection = ee.ImageCollection("ESA/WorldCover/v100")
esaLc = imageCollection.filterBounds(aoi).first().clip(aoi)

stratifiedSamples = esaLc.stratifiedSample(**{
    "numPoints": 5,
    "region": aoi,
    "scale": 100, 
    "seed": 42, 
    "tileScale": 4, 
    "geometries": True
}).map(h.set_id)

### 4 - Display in map

**Execute only**

In [ ]:
# intialize Map
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.centerObject(aoi,6)

# add layers
Map.addLayer(aoi, {}, 'Area of interest')
Map.addLayer(esaLc, False, 'ESA Worldcover 2020')
Map.addLayer(stratifiedSamples, {}, 'Stratified Samples per class from ESA LC', False)
Map.addLayer(grid_fc, {}, 'Grid')
Map.addLayer(centerInCell, {}, '1 Center Sample per gridcell (Systematic)', False)
Map.addLayer(randomInCell, {}, '1 Random Sample per gridcell (Systematic random)', True)
#
Map

### 5 - Add auxillary data from global datasets

**Execute only**

In [ ]:
point_with_global_products = h.sample_global_products(centerInCell, outfile)
point_with_global_products